In [3]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'gay',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI'
]

In [4]:
import json
import random

In [5]:
data = []

with open('/home/husein/ssd3/ctranslate2/function-call.translated.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        texts = []
        for no, d in enumerate(l['conversations']):
            
            if d['role'] == 'user':
                t = '<manusia>: '
                if random.random() > 0.5:
                    c = 'content'
                else:
                    c = 'content_ms'
            else:
                if 'functioncall' in d['content']:
                    c = 'content'
                else:
                    if 'functioncall' in l['conversations'][no - 1]['content']:
                        continue
                    c = 'content_ms'
                t = '<bot>: '

            if d[c] is None:
                break
            
            if 'functioncall' in d[c]:
                d[c] = d[c].replace('FUNCTION RESPONSE:', '<functioncall>')

            n = d[c]
            n = n.strip()
            if len(n) < 3:
                break
            t = t + n
            texts.append(t)
            
        if not len(texts):
            continue
        while len(texts) and texts[-1].startswith('<manusia>'):
            texts = texts[:-1]
        if not len(texts):
            continue
        if not texts[0].startswith('<manusia>'):
            continue
        
        data.append({
            'prompt_input': None,
            'input': '\n'.join(texts),
            'output': None,
            'function_call': l['function_call'],
        })

In [40]:
def generate_and_tokenize_prompt(row):
    texts = ['<s>']

    if 'function_call' in row:
        t = row['function_call']
        texts.append(f'\n[FUNCTIONCALL]\n{t}\n')

    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human.strip())
            outputs.append(bot.strip())
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    for u, a in zip(inputs, outputs):
        texts.append(f'[INST] {u.strip()} [/INST]{a.strip()}</s> ')

    prompt = ''.join(texts)
    return {'text': prompt}

In [58]:
data = []
with open('shuf-combine-alternate-no-alignment-malay-multitasks.jsonl') as fopen:
    for l in fopen:
        data.append(json.loads(l))
        if len(data) == 20:
            break

In [62]:
data[-4]

{'prompt_input': None,
 'input': '<manusia>: Can you describe the flora and fauna found on the highest point in Iowa?\n<bot>: Titik tertinggi di Iowa ialah Hawkeye Point, yang terletak di Daerah Osceola. Kawasan sekitar Hawkeye Point adalah terutamanya tanah ladang, dan tidak banyak tumbuh-tumbuhan semula jadi atau hidupan liar di puncak itu sendiri. Walau bagaimanapun, Daerah Osceola dan kawasan sekitarnya adalah rumah kepada pelbagai jenis tumbuhan dan haiwan.\n\nTumbuhan yang biasa ditemui di kawasan itu termasuk tanaman seperti jagung dan kacang soya, serta rumput seperti bunga liar dan rumput padang rumput. Osceola County terkenal dengan tanah lembapnya, yang menyokong pelbagai tumbuhan akuatik seperti cattails, padi liar dan teratai air.\n\nFauna yang terdapat di kawasan itu adalah pelbagai dan termasuk pelbagai haiwan seperti rusa ekor putih, coyote, musang merah, rakun, skunk dan opossum. Daerah ini juga merupakan rumah kepada pelbagai spesies burung, termasuk helang botak, bur

In [63]:
generate_and_tokenize_prompt(data[-4])

{'text': '<s>[INST] Can you describe the flora and fauna found on the highest point in Iowa? [/INST]Titik tertinggi di Iowa ialah Hawkeye Point, yang terletak di Daerah Osceola. Kawasan sekitar Hawkeye Point adalah terutamanya tanah ladang, dan tidak banyak tumbuh-tumbuhan semula jadi atau hidupan liar di puncak itu sendiri. Walau bagaimanapun, Daerah Osceola dan kawasan sekitarnya adalah rumah kepada pelbagai jenis tumbuhan dan haiwan.\n\nTumbuhan yang biasa ditemui di kawasan itu termasuk tanaman seperti jagung dan kacang soya, serta rumput seperti bunga liar dan rumput padang rumput. Osceola County terkenal dengan tanah lembapnya, yang menyokong pelbagai tumbuhan akuatik seperti cattails, padi liar dan teratai air.\n\nFauna yang terdapat di kawasan itu adalah pelbagai dan termasuk pelbagai haiwan seperti rusa ekor putih, coyote, musang merah, rakun, skunk dan opossum. Daerah ini juga merupakan rumah kepada pelbagai spesies burung, termasuk helang botak, burung biru timur, wrens ruma

In [41]:
from tqdm import tqdm

for d in data:
    generate_and_tokenize_prompt(d)['text']

In [42]:
print(generate_and_tokenize_prompt(data[4])['text'])

<s>
[FUNCTIONCALL]
{
    "name": "calculate_median",
    "description": "Calculate the median of a list of numbers",
    "parameters": {
        "type": "object",
        "properties": {
            "numbers": {
                "type": "array",
                "items": {
                    "type": "number"
                },
                "description": "A list of numbers"
            }
        },
        "required": [
            "numbers"
        ]
    }
}
[INST] Hi, I have a list of numbers and I need to find the median. Can you help me with that? [/INST]Tentu saja, saya boleh membantu anda dengan itu. Sila berikan saya senarai nombor.</s> [INST] Nombornya ialah 5, 2, 9, 1, 7, 4, 6, 3, 8. [/INST]<functioncall> {"name": "calculate_median", "arguments": '{"numbers": [5, 2, 9, 1, 7, 4, 6, 3, 8]}'} 


<functioncall> {"median": 5}</s> 


In [27]:
splitted = d['input'].split('<bot>:')
splitted

['<manusia>: Hai, saya perlu mengira bayaran pinjaman bulanan saya. Saya mempunyai pinjaman sebanyak $50000 dengan kadar faedah tahunan sebanyak 5% dan tempoh pinjaman selama 10 tahun. Bolehkah anda membantu saya dengan itu?\n',
 ' Tentu saja, saya boleh membantu anda dengan itu. Biar saya mengira untuk anda.\n',
 ' <functioncall> {"name": "calculate_loan_payment", "arguments": \'{"principal": 50000, "interest_rate": 5, "loan_term": 10}\'} \n\n\n<functioncall> {"monthly_payment": "$530.33"}\n<manusia>: That\'s great! Can you also help me book a flight to New York next week?\n',
 ' Saya minta maaf, tetapi sebagai AI, saya tidak mempunyai keupayaan untuk menempah penerbangan atau melakukan tugas luar. Fungsi saya sekarang adalah untuk mengira pembayaran pinjaman. Saya mengesyorkan menggunakan tapak web atau aplikasi tempahan perjalanan untuk keperluan tempahan penerbangan anda.']

In [16]:
print(generate_and_tokenize_prompt(data[1])['text'])

<s>
[FUNCTIONCALL] {
    "name": "get_news_headlines",
    "description": "Get the latest news headlines",
    "parameters": {
        "type": "object",
        "properties": {
            "country": {
                "type": "string",
                "description": "The country for which to fetch news"
            }
        },
        "required": [
            "country"
        ]
    }
} [INST] Bolehkah anda memberitahu saya tajuk berita terkini untuk Amerika Syarikat? [/INST]<functioncall> {"name": "get_news_headlines", "arguments": '{"country": "United States"}'} 


<functioncall> {"headlines": ["Biden announces new vaccine mandates", "Hurricane Ida devastates Louisiana", "Apple unveils new iPhone", "NASA's Perseverance rover collects first Mars rock sample"]}</s> [INST] Itu menarik. Bagaimana dengan berita di Perancis? [/INST]<functioncall> {"name": "get_news_headlines", "arguments": '{"country": "France"}'} 


<functioncall> {"headlines": ["France recalls ambassadors to US and Aus

In [67]:
import json

with open('prepared-glaive-function-call.jsonl', 'w') as fopen:
    for l in data:
        fopen.write(f'{json.dumps(l)}\n')